# CS 109A/AC 209A/STAT 121A Data Science: Midterm 2
**Harvard University**<br>
**Fall 2016**<br>
**Instructors: W. Pan, P. Protopapas, K. Rader**<br>
**Due Date: ** Tuesday, November 22nd, 2016 at 12:00pm

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn import preprocessing
from sklearn.cross_validation import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier as DecisionTree
from sklearn.ensemble import RandomForestClassifier as RandomForest
from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split as sk_split
from sklearn.linear_model import LogisticRegressionCV as LogRegCV
from sklearn import ensemble
import matplotlib
import matplotlib.pyplot as plt
import collections
%matplotlib inline

# Part I: Diagnosing the Semian Flu 2016

You are given the early data for an outbreak of a dangerous virus originating from a group of primates being keeped in a Massechussetts biomedical research lab, this virus is dubbed the "Semian Flu".

You have the medical records of $n$ number of patients in `'flu_train.csv`. There are two general types of patients in the data, flu patients and healthy (this is recorded in the column labeled `flu`, a 0 indicates the absences of the virus and a 1 indicates presence). Furthermore, scientists have found that there are two strains of the virus, each requiring a different type of treatment (this is recorded in the column labeled `flutype`, a 1 indicates the absences of the virus, a 1 indicates presence of strain 1 and a 2 indicates the presence of strain 2).

**Your task:** build a model to predict if a given patient has the flu. Your goal is to catch as many flu patients as possible without misdiagnosing too many healthy patients.

**The deliverable:** a function called `flu_predict` which satisfies:

- input: `x_test`, a set of medical predictors for a group of patients
- output: `y_pred`, a set of labels, one for each patient; 0 for healthy and 1 for infected with the flu virus

The MA state government will use your model to diagnose sets of future patients (held by us). You can expect that there will be an increase in the number of flu patients in any groups of patients in the future.

We provide you with some benchmarks for comparison.

**Baseline Model:** 
- ~50% expected accuracy on healthy patients in observed data
- ~50% expected accuracy on flu patients in observed data
- ~50% expected accuracy on healthy patients in future data 
- ~50% expected accuracy on flu patients in future data
- time to build: 5 min

**Reasonable Model:** 
- ~69% expected accuracy on healthy patients in observed data
- ~55% expected accuracy on flu patients, in observed data
- ~69% expected accuracy on healthy patients in observed data
- ~60% expected accuracy on flu patients, in observed data
- time to build: 20 min

**Grading:**
Your grade will be based on:
1. your model's ability to out-perform our benchmarks
2. your ability to carefully and thoroughly follow the data science pipeline (see lecture slides for definition)
3. the extend to which all choices are reasonable and defensible by methods you have learned in this class

----

Don't know what exactly is meant by "data science pipeline" is - I had previously merged all the lecture sides into two large pdf's, splitting the course in two essentially and the only location I can find the text "pipe" is in the very first lecture and it is only a side reference. I guess it could have been in one of the html "slides", didn't have time to digging through those. I wanted to try out some visualizations but didn't have time - and by that point they would have been added in after everything was done, somewhat defeating the purpose.  

My background is currently in data engineering and as you'll see below, the source data is my biggest concernt. For Midterm I the details were fantastical and there wasn't anything to really be done with the data anyway. Now that we have a more realistic example, and we've done more discussion of ethics etc. in class, the real answer to this question is to of course 1) void my contract with the state of MA, and 2) move out of MA cause state officials don't know what they are doing in hiring me. But more seriously, I truly believe that at least 80% of the work below should have been concentrated on reviewing and cleaning the data vs. actually creating the models. That estimate is given my lack of domain knowledge - I suspect with more background in the biostatistics world I should spend even more time on data prep and cleaning. Anyway, I spent some time initial on the data side but decided I couldn't do anything too worthwhile there given time constraints and moved on to the more basic tasks requested.


In [2]:
# load up the two datasets and inspect
train = pd.read_csv('flu_train.csv', low_memory=False)  # low memory is set false for better type inference
train.head()

,ID,Gender,Age,AgeDecade,AgeMonths,Race1,Race3,Education,MaritalStatus,HHIncome,...,HardDrugs,SexEver,SexAge,SexNumPartnLife,SexNumPartYear,SameSex,SexOrientation,PregnantNow,flu,flutype
0,51624,male,34,30-39,409,White,NaN,High School,Married,25000-34999,...,Yes,Yes,16,8,1,No,Heterosexual,NaN,0,1
1,51630,female,49,40-49,596,White,NaN,Some College,LivePartner,35000-44999,...,Yes,Yes,12,10,1,Yes,Heterosexual,NaN,0,1
2,51638,male,9,0-9,115,White,NaN,NaN,NaN,75000-99999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
3,51646,male,8,0-9,101,White,NaN,NaN,NaN,55000-64999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
4,51647,female,45,40-49,541,White,NaN,College Grad,Married,75000-99999,...,No,Yes,13,20,0,Yes,Bisexual,NaN,0,1


In [3]:
test = pd.read_csv('flu_test_no_y.csv', low_memory=False)  # low memory is set false for better type inference
test.head()

,ID,Gender,Age,AgeDecade,AgeMonths,Race1,Race3,Education,MaritalStatus,HHIncome,...,RegularMarij,AgeRegMarij,HardDrugs,SexEver,SexAge,SexNumPartnLife,SexNumPartYear,SameSex,SexOrientation,PregnantNow
0,51625,male,4,0-9,49,Other,NaN,NaN,NaN,20000-24999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,51678,male,60,60-69,721,White,NaN,High School,Married,15000-19999,...,NaN,NaN,No,Yes,20,1,NaN,No,NaN,NaN
2,51694,male,38,30-39,458,White,NaN,Some College,Married,20000-24999,...,No,NaN,No,Yes,23,1,1,No,Heterosexual,NaN
3,51695,male,8,0-9,104,White,NaN,NaN,NaN,65000-74999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,51711,female,59,50-59,718,Other,NaN,8th Grade,Widowed,20000-24999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:

train_size = train.shape[0]
test_size = test.shape[0]
print 'Size of training set: {:,}'.format(train_size)
print 'Size of test set: {:,}'.format(test_size)
print 'Train/test split: {:.0f}/{:.0f}'.format(
    train_size*1.0/(train_size+test_size)*100, 
    test_size*1.0/(train_size+test_size)*100)
print
print 'Train/test split is a little more extreme than in some cases but seems reasonable. Given that the split is favored'
print ' on the train side, we could potentially remove the two predicted Y values from a subset of train and move to test'
print ' but should be ok as-is.'

Size of training set: 5,246
Size of test set: 1,533
Train/test split: 77/23

Train/test split is a little more extreme than in some cases but seems reasonable. Given that the split is favored
 on the train side, we could potentially remove the two predicted Y values from a subset of train and move to test
 but should be ok as-is.


In [5]:
train_columns = train.columns.values
print 'train has {} columns'.format(len(train_columns))
train_columns

train has 76 columns


array(['ID', 'Gender', 'Age', 'AgeDecade', 'AgeMonths', 'Race1', 'Race3',
       'Education', 'MaritalStatus', 'HHIncome', 'HHIncomeMid', 'Poverty',
       'HomeRooms', 'HomeOwn', 'Work', 'Weight', 'Length', 'HeadCirc',
       'Height', 'BMI', 'BMICatUnder20yrs', 'BMI_WHO', 'Pulse', 'BPSysAve',
       'BPDiaAve', 'BPSys1', 'BPDia1', 'BPSys2', 'BPDia2', 'BPSys3',
       'BPDia3', 'Testosterone', 'DirectChol', 'TotChol', 'UrineVol1',
       'UrineFlow1', 'UrineVol2', 'UrineFlow2', 'Diabetes', 'DiabetesAge',
       'HealthGen', 'DaysMentHlthBad', 'LittleInterest', 'Depressed',
       'nPregnancies', 'nBabies', 'Age1stBaby', 'SleepHrsNight',
       'SleepTrouble', 'PhysActive', 'PhysActiveDays', 'TVHrsDay',
       'CompHrsDay', 'TVHrsDayChild', 'CompHrsDayChild', 'Alcohol12PlusYr',
       'AlcoholDay', 'AlcoholYear', 'SmokeNow', 'Smoke100', 'Smoke100n',
       'SmokeAge', 'Marijuana', 'AgeFirstMarij', 'RegularMarij',
       'AgeRegMarij', 'HardDrugs', 'SexEver', 'SexAge', 'SexNumPartnLife'

In [6]:
test_columns = test.columns.values
print 'test has {} columns, as expected same count as train -2 for the two predicotr columns'.format(len(test_columns))
test_columns

test has 74 columns, as expected same count as train -2 for the two predicotr columns


array(['ID', 'Gender', 'Age', 'AgeDecade', 'AgeMonths', 'Race1', 'Race3',
       'Education', 'MaritalStatus', 'HHIncome', 'HHIncomeMid', 'Poverty',
       'HomeRooms', 'HomeOwn', 'Work', 'Weight', 'Length', 'HeadCirc',
       'Height', 'BMI', 'BMICatUnder20yrs', 'BMI_WHO', 'Pulse', 'BPSysAve',
       'BPDiaAve', 'BPSys1', 'BPDia1', 'BPSys2', 'BPDia2', 'BPSys3',
       'BPDia3', 'Testosterone', 'DirectChol', 'TotChol', 'UrineVol1',
       'UrineFlow1', 'UrineVol2', 'UrineFlow2', 'Diabetes', 'DiabetesAge',
       'HealthGen', 'DaysMentHlthBad', 'LittleInterest', 'Depressed',
       'nPregnancies', 'nBabies', 'Age1stBaby', 'SleepHrsNight',
       'SleepTrouble', 'PhysActive', 'PhysActiveDays', 'TVHrsDay',
       'CompHrsDay', 'TVHrsDayChild', 'CompHrsDayChild', 'Alcohol12PlusYr',
       'AlcoholDay', 'AlcoholYear', 'SmokeNow', 'Smoke100', 'Smoke100n',
       'SmokeAge', 'Marijuana', 'AgeFirstMarij', 'RegularMarij',
       'AgeRegMarij', 'HardDrugs', 'SexEver', 'SexAge', 'SexNumPartnLife'

In [7]:
print 'columns in train but not in test: {}'.format(
    ', '.join([col for col in set(train_columns).difference(set(test_columns))]) )
print 'columns in test but not in train: {}'.format(
    ', '.join([col for col in set(test_columns).difference(set(train_columns))]) )
print
print 'ok, confirmed, looks like indeed those two flu dependent variables are the only difference'

columns in train but not in test: flutype, flu
columns in test but not in train: 

ok, confirmed, looks like indeed those two flu dependent variables are the only difference


In [8]:
# take a look to see which variables are missing a lot of data
# create a missing_prop column to hold proportion that are missing values in a given column
df_count = pd.DataFrame( train.count())
df_count['missing_prop'] = df_count.apply(lambda cnt: 1-(cnt*1.0/train_size))
df_count


,0,missing_prop
ID,5246,0.000000
Gender,5246,0.000000
Age,5246,0.000000
AgeDecade,5057,0.036027
AgeMonths,2726,0.480366
Race1,5246,0.000000
Race3,2508,0.521921
Education,3574,0.318719
MaritalStatus,3580,0.317575
HHIncome,4798,0.085398


In [9]:
# take a look at columns missing data in > 50% of the observations
df_missing_50 = df_count[df_count['missing_prop']>0.50]
df_missing_50

,0,missing_prop
Race3,2508,0.521921
Length,356,0.932139
HeadCirc,61,0.988372
BMICatUnder20yrs,724,0.861990
Testosterone,2008,0.617232
UrineVol2,763,0.854556
UrineFlow2,763,0.854556
DiabetesAge,330,0.937095
nPregnancies,1283,0.755433
nBabies,1182,0.774685


Looking at those missing the most data, first thing that comes to mind that there is no way I should in anyway be involved in this part of the pipeline. I lack the necessary domain knowledge and proceeding forward would be extremely reckless.  

Now I will proceed forward.  

A data dictionary of some kind would also be helpful, but I'll guess at the columns' meaning.  
Actually, any kind of analysis of above, and those that meet a higher threshold of missing data, will take a couple of hours at least, leaving little time for the rest of the midterm, so I guess that isn't an option.


And after a little deeper review of the data, my concerns are much deeper. Simply on that level it would be unwise to proceed - given an estimate of 6 hours for the midterm, I would think at least 4 should be spent on analyzing the data, much less any kind of imputation or filling missing values, and that is given my lack of domain knowledge. I'll probably wind up ruthlessly culling columns rather than going trying to fill any missing values.  
  
I consider pregnancy related columns to be one of the more important pieces of data that should be carefully considered, all things considered (effect on mother's overall health, effect on fetus health, term of pregnancy, )

In [10]:
woman_childbearing_age = train[(train['Gender']=='female') & (train['Age'] > 14) & (train['Age'] < 50)]
num_woman_childbearing_age = woman_childbearing_age.shape[0]
num_woman_childbearing_age

1236

In [11]:
print 'what percent of rows are really "missing" PregnantNow values'
print '{:.2f}%'.format((1-woman_childbearing_age['PregnantNow'].count()*1.0/num_woman_childbearing_age)*100)

what percent of rows are really "missing" PregnantNow values
28.56%


Columns for first pass
- Age, is 100% filled, get rid of Age Decade, covered by Age, & AgeMonths, thats just going to add noise
- Race1, Race3
Socialogical factors that migh affect flu infection rates
- Marital Status
- HHIncome
- Poverty
- HHIncomeMid
- HomeOwn
- Work
Bio factors
- Weight
- Height
- BMI
- Pulse
- various BP
    - Pulse	BPSysAve	BPDiaAve	BPSys1	BPDia1	BPSys2	BPDia2	BPSys3	BPDia3
    
Ok, this is silly, I'll just list the columns I'm going to use and try to fill in some justification later if time. Several relate to the idea that if someone has the flu they won't feel well.

  
ID  
Gender  
Age  
Race1  
Race3  
Education  
MaritalStatus  
HHIncome  
Poverty  
HomeOwn  
Work  
Weight  
Height  
BMI  
Pulse  
BPSysAve  
BPDiaAve  
BPSys1  
BPDia1  
BPSys2  
BPDia2  
BPSys3  
BPDia3  
DirectChol  
TotChol  
UrineVol1  
UrineFlow1  
UrineVol2  
UrineFlow2  
Diabetes  
HealthGen  
DaysMentHlthBad  
Depressed  
SleepHrsNight  
SleepTrouble  
PhysActive  
PhysActiveDays  
SmokeNow  
Marijuana  
HardDrugs  
PregnantNow  
  
  
 

In [12]:
keep_columns = ['ID','Gender','Age','Race1','Race3','Education','MaritalStatus','HHIncome','Poverty','HomeOwn','Work',
              'Weight','Height','BMI','Pulse','BPSysAve','BPDiaAve','BPSys1','BPDia1','BPSys2','BPDia2','BPSys3','BPDia3',
              'DirectChol','TotChol','UrineVol1','UrineFlow1','UrineVol2','UrineFlow2','Diabetes','HealthGen',
              'DaysMentHlthBad','Depressed','SleepHrsNight','SleepTrouble','PhysActive','PhysActiveDays','SmokeNow',
              'Marijuana','HardDrugs','PregnantNow']

train = train[keep_columns + ['flu','flutype']]

In [13]:
df_count = pd.DataFrame( train.count())
df_count['missing_prop'] = df_count.apply(lambda cnt: 1-(cnt*1.0/train_size))

missing_again = df_count[df_count['missing_prop'] > .50]
missing_again

,0,missing_prop
Race3,2508,0.521921
UrineVol2,763,0.854556
UrineFlow2,763,0.854556
PhysActiveDays,2374,0.547465
SmokeNow,1539,0.706634
Marijuana,2411,0.540412
PregnantNow,883,0.831681


In [14]:
drop_columns = list(missing_again.index.values)

train = train.drop(drop_columns, axis=1)


In [15]:
final_columns = train.columns.values

print 'left with {} columns'.format(len(train.columns.values))
print
print final_columns


left with 36 columns

['ID' 'Gender' 'Age' 'Race1' 'Education' 'MaritalStatus' 'HHIncome'
 'Poverty' 'HomeOwn' 'Work' 'Weight' 'Height' 'BMI' 'Pulse' 'BPSysAve'
 'BPDiaAve' 'BPSys1' 'BPDia1' 'BPSys2' 'BPDia2' 'BPSys3' 'BPDia3'
 'DirectChol' 'TotChol' 'UrineVol1' 'UrineFlow1' 'Diabetes' 'HealthGen'
 'DaysMentHlthBad' 'Depressed' 'SleepHrsNight' 'SleepTrouble' 'PhysActive'
 'HardDrugs' 'flu' 'flutype']


In [16]:
#TODO impute missing values ha ha, KNN is the only proper way, mean etc. all imply domain knowledge


# for now, do this so things can actually finish... and anything fancier would need to be done on test set also
# this is going to have to do, at the least want to turn 'DaysMentHlthBad' into some kind of binary or binned value
train = train.fillna(train.mean())


In [17]:
# check the datatypes of remaining columns
train.dtypes

ID                   int64
Gender              object
Age                  int64
Race1               object
Education           object
MaritalStatus       object
HHIncome            object
Poverty            float64
HomeOwn             object
Work                object
Weight             float64
Height             float64
BMI                float64
Pulse              float64
BPSysAve           float64
BPDiaAve           float64
BPSys1             float64
BPDia1             float64
BPSys2             float64
BPDia2             float64
BPSys3             float64
BPDia3             float64
DirectChol         float64
TotChol            float64
UrineVol1          float64
UrineFlow1         float64
Diabetes            object
HealthGen           object
DaysMentHlthBad    float64
Depressed           object
SleepHrsNight      float64
SleepTrouble        object
PhysActive          object
HardDrugs           object
flu                  int64
flutype              int64
dtype: object

In [18]:
# modified lab 10 code to encode the categorical variables via one-hot encoding

#Encode categorical variables
def encode_categorical(array):
    if not array.dtype == np.dtype('float64'):
        return preprocessing.LabelEncoder().fit_transform(array) 
    else:
        return array
    

def prep_data(df, y_col_count):
    to_float = df.dtypes[df.dtypes == 'int64'].index

    # oops, this was a mistake in my first run, where not doing to_float resulted in better model
    # Converted columns to floating point
    for feature_name in to_float:
        #if feature_name != 'ID':
        df[feature_name] = df[feature_name].astype(float)

    # Categorical columns for use in one-hot encoder
    categorical = (df.dtypes.values != np.dtype('float64'))    
    df = df.apply(encode_categorical)

    if y_col_count:
        # Get numpy array from data, we have two flu indicators in train
        x = df.values[:, :-y_col_count]
        y = df.values[:, -y_col_count:]
    else:
        x = df.values
        y = None
        
    #print 'x.shape:',x.shape
        
    # Apply one hot endcoing
    encoder = preprocessing.OneHotEncoder(categorical_features=categorical, sparse=False)  # Last value in mask is y
    x = encoder.fit_transform(x)
    
    return x, y

x_train, y_train = prep_data(train, y_col_count=2)
#x_test, y_test = prep_data(test, y_col_count=0)

y_train_p1 = y_train[: , 0]
y_train_p2 = y_train[: , 1]


C:\Anaconda\lib\site-packages\numpy\lib\arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))
C:\Anaconda\lib\site-packages\sklearn\preprocessing\data.py:1635: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 34 but corresponding boolean dimension is 36
  sel[np.asarray(selected)] = True


In [19]:
# below and several code snippets afterwards are from Lab10 notebook

#Function for computing the accuracy a given model on the entire test set, the accuracy on class 0 in the test set
#and the accuracy on class 1
score = lambda model, x_test, y_test: pd.Series([model.score(x_test, y_test), 
                                                 model.score(x_test[y_test==0], y_test[y_test==0]),
                                                 model.score(x_test[y_test==1], y_test[y_test==1])],
                                                index=['overall accuracy', 
                                                       'accuracy on healthy patients', 
                                                       'accuracy on flu patients'])


In [20]:
#A model that labels everything 1
class Pos_model(object):
    def predict(self, x):
        return np.array([1] * len(x))
    def score(self, x, y):
        y_pred = self.predict(x)
        y_err = y - y_pred
        return len(y_err[y_err == 0]) * 1. / len(y_err)
    
#A model that labels everything 0
class Neg_model(object):
    def predict(self, x):
        return np.array([0] * len(x))
    
    def score(self, x, y):
        y_pred = self.predict(x)
        y_err = y - y_pred
        return len(y_err[y_err == 0]) * 1. / len(y_err)


#A model that randomly labels things
class Random_model(object):
    def predict(self, x):
        return np.random.randint(0, 2, len(x))
    
    def score(self, x, y):
        y_pred = self.predict(x)
        y_err = y - y_pred
        return len(y_err[y_err == 0]) * 1. / len(y_err)

In [21]:
pos_model = Pos_model()
pos_model_scores = score(pos_model, x_train, y_train_p1)

neg_model = Neg_model()
neg_model_scores = score(neg_model, x_train, y_train_p1)

random_model = Random_model()
random_model_scores = score(random_model, x_train, y_train_p1)

In [22]:
#Score Dataframe
score_df = pd.DataFrame({'pos model': pos_model_scores,
                         'neg model': neg_model_scores,
                         'random model': random_model_scores})
score_df

,neg model,pos model,random model
overall accuracy,0.940907,0.059093,0.497713
accuracy on healthy patients,1.000000,0.000000,0.499595
accuracy on flu patients,0.000000,1.000000,0.490323


In [23]:

print 'random baseline model, reports ~50% accuracy on healthy and on flu patients in observed data:'
print '  healthy patient accuracy {:.0f}%'.format(score_df.loc['accuracy on healthy patients'][2]*100)
print '  flu patient accuracy     {:.0f}%'.format(score_df.loc['accuracy on flu patients'][2]*100)
print 'Assuming train and test are independent samples, it is a reasonable assumption that the "Random" baseline model'
print ' perform just as well on future data.'

random baseline model, reports ~50% accuracy on healthy and on flu patients in observed data:
  healthy patient accuracy 50%
  flu patient accuracy     49%
Assuming train and test are independent samples, it is a reasonable assumption that the "Random" baseline model
 perform just as well on future data.


In [48]:
#Unweighted logistic regression
unweighted_logistic = LogisticRegression()
unweighted_logistic.fit(x_train, y_train_p1)

unweighted_log_scores = score(unweighted_logistic, x_train, y_train_p1)
print 'unweighted log'


#Weighted logistic regression
weighted_logistic = LogisticRegression(class_weight='balanced')
weighted_logistic.fit(x_train, y_train_p1)

weighted_log_scores = score(weighted_logistic, x_train, y_train_p1)
print 'weighted log'

#LDA
lda = LDA()
lda.fit(x_train, y_train_p1)

lda_scores = score(lda, x_train, y_train_p1)
print 'lda'

#QDA
qda = QDA()
qda.fit(x_train, y_train_p1)

qda_scores = score(qda, x_train, y_train_p1)
print 'qda'

#Decision Tree
tree = DecisionTree(max_depth=6)
tree.fit(x_train, y_train_p1)

tree_scores = score(tree, x_train, y_train_p1)
print 'tree'

#Random Forest
rf = RandomForest()
rf.fit(x_train, y_train_p1)

rf_scores = score(rf, x_train, y_train_p1)

print 'rf'

knn=KNN(25)
knn.fit(x_train, y_train_p1)
knn_scores = score(knn, x_train, y_train_p1)

print 'knn'



unweighted log
weighted log
lda
qda
tree
rf
knn


In [25]:
#Score Dataframe
score_df = pd.DataFrame({#'knn': knn_scores, 
                         'unweighted logistic': unweighted_log_scores,
                         'weighted logistic': weighted_log_scores,
                         'lda': lda_scores,
                         'qda': qda_scores,
                         'tree': tree_scores,
                         'rf': rf_scores,
                         'knn': knn_scores})
score_df

,knn,lda,qda,rf,tree,unweighted logistic,weighted logistic
overall accuracy,0.940907,0.943195,0.403736,0.990469,0.955204,0.941860,0.744186
accuracy on healthy patients,1.000000,0.987034,0.367504,1.000000,0.999797,1.000000,0.746759
accuracy on flu patients,0.000000,0.245161,0.980645,0.838710,0.245161,0.016129,0.703226


In [26]:
print 'Weighted logistic regression had best accuracy, considering each of the two patient classes'
#Weighted logistic regression
weighted_logistic = LogisticRegression(class_weight='balanced')
weighted_logistic.fit(x_train, y_train_p1)

weighted_log_scores = score(weighted_logistic, x_train, y_train_p1)
weighted_log_scores

Weighted logistic regression had best accuracy, considering each of the two patient classes


overall accuracy                0.744186
accuracy on healthy patients    0.746759
accuracy on flu patients        0.703226
dtype: float64

In [27]:

# pull a validation set out of the 5k records in our train set, about 1600 records
x_train_train, x_valid, y_train_train_p1, y_valid_p1 = sk_split(x_train, y_train_p1, test_size=0.30)

print 'sanity check the new split sizes'
print x_train.shape, y_train_p1.shape
print x_train_train.shape, y_train_train_p1.shape, x_valid.shape, y_valid_p1.shape


sanity check the new split sizes
(5246L, 84L) (5246L,)
(3672L, 84L) (3672L,) (1574L, 84L) (1574L,)


In [28]:

# I did try some tuning parameters on the logistic regression, using LogRegCV model but everything
# I tried wound up doing worse when looking at accuracy within each class
def nothing_doing():

    #Generate array of L2 regularization parameters
    regularization = 10.**np.arange(-10, 5)

    #Fit logistic model with cross validation to select the optimal regularization parameter
    logistic = LogRegCV(cv=5, 
                        penalty='l2', 
                        Cs=regularization, 
                        solver='liblinear', 
                        n_jobs=-1,
                        class_weight='balanced')
    #### all of train: logistic.fit(x_train, y_train_p1)
    logistic.fit(x_train_train, y_train_train_p1)



In [29]:
# standard model, but train on new train & use out-of-sample validation set to score

regularization = 10.**np.arange(-5, 5)

for c in regularization:
    weighted_logistic = LogisticRegression(class_weight='balanced', C=c)
    weighted_logistic.fit(x_train_train, y_train_train_p1)

    weighted_log_scores = score(weighted_logistic, x_valid, y_valid_p1)
    print c
    print weighted_log_scores

#overall accuracy                0.741423
#accuracy on healthy patients    0.747305
#accuracy on flu patients        0.644444

1e-05
overall accuracy                0.702668
accuracy on healthy patients    0.710331
accuracy on flu patients        0.580645
dtype: float64
0.0001
overall accuracy                0.710928
accuracy on healthy patients    0.722485
accuracy on flu patients        0.526882
dtype: float64
0.001
overall accuracy                0.736341
accuracy on healthy patients    0.742066
accuracy on flu patients        0.645161
dtype: float64
0.01
overall accuracy                0.722363
accuracy on healthy patients    0.725186
accuracy on flu patients        0.677419
dtype: float64
0.1
overall accuracy                0.708386
accuracy on healthy patients    0.707630
accuracy on flu patients        0.720430
dtype: float64
1.0
overall accuracy                0.707116
accuracy on healthy patients    0.707630
accuracy on flu patients        0.698925
dtype: float64
10.0
overall accuracy                0.705210
accuracy on healthy patients    0.705604
accuracy on flu patients        0.698925
dtype: float

In [30]:
# for final model, go with C=10000
final_log_model = LogisticRegression(class_weight='balanced', C=10000)
final_log_model.fit(x_train_train, y_train_train_p1)

final_log_scores = score(final_log_model, x_valid, y_valid_p1)
print final_log_scores


overall accuracy                0.705210
accuracy on healthy patients    0.705604
accuracy on flu patients        0.698925
dtype: float64


cv=5 (or 10), liblinear, balanced :
    
overall accuracy                0.745235
accuracy on healthy patients    0.751020
accuracy on flu patients        0.663462


In [31]:
#early test, replaced with version that has all written out

def flu_predict_pass1(x_test):
    cols = [col for col in x_test.columns if col in final_columns]
    x_test = x_test[cols]
    x_test = x_test.fillna(test.mean())
    x_test_prepped, y_test = prep_data(x_test, y_col_count=0)
    
    pred = final_log_model.predict(x_test_prepped)
    
    return pred
    
#final_log_model.predict(x_test)
#x_test, y_test = prep_data(test, y_col_count=0)

predictions = flu_predict_pass1(test)
predictions[:10]

array([ 0.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.])

In [32]:
def flu_predict(x_test):
    
    #Encode categorical variables
    def encode_categorical(array):
        if not array.dtype == np.dtype('float64'):
            return preprocessing.LabelEncoder().fit_transform(array) 
        else:
            return array
    
    cols = [col for col in x_test.columns if col in final_columns]
    x_test = x_test[cols]
    x_test = x_test.fillna(test.mean()) 

    to_float = x_test.dtypes[x_test.dtypes == 'int64'].index
    # Converted columns to floating point
    for feature_name in to_float:
        #if feature_name != 'ID':
        x_test[feature_name] = x_test[feature_name].astype(float)

    # Categorical columns for use in one-hot encoder
    categorical = (x_test.dtypes.values != np.dtype('float64'))    
    x_test = x_test.apply(encode_categorical)
   
    x = x_test.values

    # Apply one hot endcoing
    encoder = preprocessing.OneHotEncoder(categorical_features=categorical, sparse=False)  # Last value in mask is y
    x = encoder.fit_transform(x)

    pred = final_log_model.predict(x)
    
    return pred



#predictions = flu_predict(test)
#predictions[:10]


In [33]:
# make predictions using above function, take a peek at per-flu counts
predictions = flu_predict(test)
print collections.Counter(predictions)

# write to file
out = np.column_stack((test['ID'].values, predictions))
np.savetxt('p1_out.csv', out, delimiter=',', header='ID,flu', comments='')

back_in = pd.read_csv('p1_out.csv', low_memory=False)  
back_in.head()


Counter({0.0: 1010, 1.0: 523})


,ID,flu
0,51625,0
1,51678,1
2,51694,1
3,51695,1
4,51711,1


# Part II: Diagnosing Strains of the Semian Flu

From a public health perspective, we want to balance the cost of vaccinations, early interventions and the cost of treating flu complications of unvaccinated people. 

There are two different strains of the flu: strain 1 has a cheaper early intervention as well as a cheaper treatment for flu complications, but patients with strain 1 has a higher rate of developing complications if treated with the wrong intervention. Strain 2 has a more expensive early intervention as well as a more costly treatment for flu complications, but patients with strain 2 has a lower rate of developing complications if treated with the wrong intervention. With no intervention, flu patients develop complications at the same rate regardless of the strain. 

**Your task:** build a model to predict if a given patient has the flu and identify the flu strain. The state government of MA will use your model to inform public health policies: we will vaccinate people you've identified as healthy and apply corresponding interventions to patients with different strains of the flu. We have provided you with a function to compute the total expected cost of this policy decision that takes into account the cost of the vaccine, the interventions and the cost of the treatments for flu complications resulting from misdiagnosing patients. Your goal is to make sure your model produces a public health policy with the lowest associated expected cost.

**The deliverable:** a function called `flu_predict` which satisfies:

- input: `x_test`, a set of medical predictors for a group of patients
- output: `y_pred`, a set of labels, one for each patient; 1 for healthy, 2 for infected with strain 1, and 3 for infected with strain 2.

The MA state government will use your model to diagnose sets of future patients (held by us). You can expect that there will be an increase in the number of flu patients in any groups of patients in the future.

We provide you with some benchmarks for comparison.

**Three Baseline Models:** 
- expected cost on observed data: \$6,818,206.0, \$7,035,735.0, \$8,297,197.5
- time to build: 1 min

**Reasonable Model:** 
- expected cost on observed data: $6,300,000
- time to build: 20 min

**Grading:**
Your grade will be based on:
1. your model's ability to out-perform our benchmarks
2. your ability to carefully and thoroughly follow the data science pipeline (see lecture slides for definition)
3. the extend to which all choices are reasonable and defensible by methods you have learned in this class

In [34]:
#--------  cost
# A function that computes the expected cost of the public healthy policy based on the 
# classifications generated by your model
# Input: 
#      y_true (true class labels)
#      y_pred (predicted class labels)
# Returns: 
#      total_cost (expected total cost)

def cost(y_true, y_pred):
    # set a seed, can't get repeatable results without it
    np.random.seed(109)
    cost_of_treatment_1 = 29500
    cost_of_treatment_2 = 45000
    cost_of_intervention_1 = 4150
    cost_of_intervention_2 = 4250
    cost_of_vaccine = 15
    
    prob_complications_untreated = 0.65
    prob_complications_1 = 0.30
    prob_complications_2 = 0.15
    
    trials = 1000
    
    
    intervention_cost = cost_of_intervention_1 * len(y_pred[y_pred==1]) + cost_of_intervention_2 * len(y_pred[y_pred==2])

    vaccine_cost = cost_of_vaccine * len(y_pred[y_pred==0])
    
    false_neg_1 = ((y_true == 1) & (y_pred == 2)).sum()
    false_neg_2 = ((y_true == 2) & (y_pred == 1)).sum()
    
    untreated_1 = ((y_true == 1) & (y_pred == 0)).sum()    
    untreated_2 = ((y_true == 2) & (y_pred == 0)).sum()
    
    false_neg_1_cost = np.random.binomial(1, prob_complications_1, (false_neg_1, trials)) * cost_of_treatment_1
    false_neg_2_cost = np.random.binomial(1, prob_complications_2, (false_neg_2, trials)) * cost_of_treatment_2
    untreated_1_cost = np.random.binomial(1, prob_complications_untreated, (untreated_1, trials)) * cost_of_treatment_1
    untreated_2_cost = np.random.binomial(1, prob_complications_untreated, (untreated_2, trials)) * cost_of_treatment_2
    
    false_neg_1_cost = false_neg_1_cost.sum(axis=0)
    expected_false_neg_1_cost = false_neg_1_cost.mean()
    
    false_neg_2_cost = false_neg_2_cost.sum(axis=0)
    expected_false_neg_2_cost = false_neg_2_cost.mean()
    
    untreated_1_cost = untreated_1_cost.sum(axis=0)
    expected_untreated_1_cost = untreated_1_cost.mean()
    
    untreated_2_cost = untreated_2_cost.sum(axis=0)
    expected_untreated_2_cost = untreated_2_cost.mean()
    
    total_cost = vaccine_cost + intervention_cost + expected_false_neg_1_cost + expected_false_neg_2_cost + expected_untreated_1_cost + expected_untreated_2_cost
    
    return total_cost

Don't know if I'm supposed to try to implement any baseline models but for the life of me I can't figure out the meaning of below so, I guess it isn't going to happen (supposing it was supposed to happen):   

> expected cost on observed data: \$6,818,206.0, \$7,035,735.0, \$8,297,197.5 


In [35]:
# re-set
y_train_p2_orig = y_train[: ,1]

# check out the per-flutype counts
print collections.Counter(y_train_p2_orig)
# normalize the flutype values to what cost() function expects
y_train_p2 = np.array(y_train_p2_orig)
y_train_p2 -= 1
# print out conformed counts
print collections.Counter(y_train_p2)


Counter({1.0: 4936, 2.0: 227, 3.0: 83})
Counter({0.0: 4936, 1.0: 227, 2.0: 83})


In [36]:
# run through each model, training on 2nd dependent variable, y_train_p2 aka flutype
costs = collections.OrderedDict()

#Unweighted logistic regression
unweighted_logistic = LogisticRegression()
unweighted_logistic.fit(x_train, y_train_p2)
y_pred = unweighted_logistic.predict(x_train)
mod = 'unweighted log'
costs[mod] = cost(y_train_p2, y_pred)

#Weighted logistic regression
weighted_logistic = LogisticRegression(class_weight='balanced')
weighted_logistic.fit(x_train, y_train_p2)
y_pred = weighted_logistic.predict(x_train)
mod = 'weighted log'
costs[mod] = cost(y_train_p2, y_pred)

#LDA
lda = LDA()
lda.fit(x_train, y_train_p2)
y_pred = lda.predict(x_train)
mod = 'lda'
costs[mod] = cost(y_train_p2, y_pred)

#QDA
qda = QDA()
qda.fit(x_train, y_train_p2)
y_pred = qda.predict(x_train)
mod = 'qda'
costs[mod] = cost(y_train_p2, y_pred)

#Decision Tree
tree = DecisionTree(max_depth=6)
tree.fit(x_train, y_train_p2)
y_pred = tree.predict(x_train)
mod = 'tree'
costs[mod] = cost(y_train_p2, y_pred)

#Random Forest
rf = RandomForest()
rf.fit(x_train, y_train_p2)
y_pred = rf.predict(x_train)
mod = 'rf'
costs[mod] = cost(y_train_p2, y_pred)

#KNN
knn=KNN(25)
knn.fit(x_train, y_train_p2)
y_pred = knn.predict(x_train)
mod = 'knn'
costs[mod] = cost(y_train_p2, y_pred)


for mod, total_cost in costs.iteritems():
    print '{:>20}:  ${:,}'.format(mod, total_cost)
    
print 'Done'

      unweighted log:  $6,784,932.0
        weighted log:  $6,279,829.5
                 lda:  $6,058,950.0
                 qda:  $15,005,502.0
                tree:  $5,692,421.5
                  rf:  $2,173,663.5
                 knn:  $6,859,270.5
Done


C:\Anaconda\lib\site-packages\sklearn\discriminant_analysis.py:453: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)


Of the implementations, Random Forest would appear to be the clear winner. 

In [37]:
print 'train set, get each prediction correct and cost will be: ${:,}'.format(cost(y_train_p2, y_train_p2))

train set, get each prediction correct and cost will be: $1,368,840.0


In [38]:
# adapted from HW7 solutions, do some parameter tuning 

# Parameters for tuning
n_trees = np.arange(20, 80, 20)  # Trees and depth are explored on an exponentially growing space,

depths = np.arange(2, 10)   # since it is assumed that trees and depth will add accuracy in a decaying fashion.
depths = np.arange(300, 1200, 100)   # since it is assumed that trees and depth will add accuracy in a decaying fashion.


# To keep track of the best model
best_score = 0
best_cost = 999999999

# Run grid search for model with 5-fold cross validation
print '5-fold cross validation:'

for trees in n_trees:
    for depth in depths:
        
        # Cross validation for every experiment
        k_folds = KFold(x_train.shape[0], n_folds=5, shuffle=True)
        scores = []
        costs = []
        for train_indices, validation_indices in k_folds:
            # Generate training data
            x_train_cv = x_train[train_indices]
            y_train_cv = y_train_p2[train_indices]
            # Generate validation data
            x_validate = x_train[validation_indices]
            y_validate = y_train_p2[validation_indices]
            
            # Fit random forest on training data
            model = RandomForest(n_estimators=trees, max_depth=depth)
            model.fit(x_train_cv, y_train_cv)
            # Score on validation data
            scores += [model.score(x_validate, y_validate)]
            
            y_pred = model.predict(x_validate)
            costs += [cost(y_validate, y_pred)]
        
        # Record and report accuracy
        average_score = np.mean(scores)
        average_cost = np.mean(costs)
        #print "Trees:", trees, "Depth:", depth, "(irrelevant) Score:", average_score
        print "Trees:", trees, "Depth:", depth, "Cost:", average_cost

        # Update our record of the best parameters see so far
        if average_cost < best_cost:
            best_cost = average_cost
            best_trees = trees
            best_depth = depth
            
# Fit model on entire train set using chosen number of trees and depth
final_model_p2 = RandomForest(n_estimators=best_trees, max_depth=best_depth)
final_model_p2.fit(x_train, y_train_p2)

print 'Chosen number of trees, depth:', best_trees, ',', best_depth
#print 'Test accuracy:', final_model_p2.score(x_train, y_train_p2)


# Chosen number of trees, depth: 50 , 1000

5-fold cross validation:
Trees: 20 Depth: 300 Cost: 1344374.1
Trees: 20 Depth: 400 Cost: 1346328.3
Trees: 20 Depth: 500 Cost: 1369303.1
Trees: 20 Depth: 600 Cost: 1344525.7
Trees: 20 Depth: 700 Cost: 1363439.0
Trees: 20 Depth: 800 Cost: 1347602.7
Trees: 20 Depth: 900 Cost: 1354489.0
Trees: 20 Depth: 1000 Cost: 1344525.7
Trees: 20 Depth: 1100 Cost: 1363439.0
Trees: 40 Depth: 300 Cost: 1355080.9
Trees: 40 Depth: 400 Cost: 1361303.8
Trees: 40 Depth: 500 Cost: 1354691.8
Trees: 40 Depth: 600 Cost: 1341031.8
Trees: 40 Depth: 700 Cost: 1355259.0
Trees: 40 Depth: 800 Cost: 1339049.5
Trees: 40 Depth: 900 Cost: 1365921.0
Trees: 40 Depth: 1000 Cost: 1355259.0
Trees: 40 Depth: 1100 Cost: 1339049.5
Trees: 60 Depth: 300 Cost: 1352953.2
Trees: 60 Depth: 400 Cost: 1361071.1
Trees: 60 Depth: 500 Cost: 1358822.8
Trees: 60 Depth: 600 Cost: 1348317.8
Trees: 60 Depth: 700 Cost: 1359447.5
Trees: 60 Depth: 800 Cost: 1361017.8
Trees: 60 Depth: 900 Cost: 1363121.4
Trees: 60 Depth: 1000 Cost: 1356310.8
Trees: 6

In [39]:
final_model_p2 = RandomForest(n_estimators=best_trees, max_depth=best_depth)
final_model_p2.fit(x_train, y_train_p2)

# lets look at cost of that final model
y_pred_model = final_model_p2.predict(x_train)
print 'best_trees: {}  best_depth: {}'.format(best_trees, best_depth)
print '{:,}'.format(cost(y_train_p2, y_pred_model))

# take a look at the per-flutype counts
print collections.Counter(y_pred_model)


best_trees: 40  best_depth: 800
1,458,696.5
Counter({0.0: 4942, 1.0: 221, 2.0: 83})


In [40]:
print 'until a late breaking change I made somewhere, 50, 1000 were the parameters that were working best'
print ' the cost was notably lower than what I am coming up with now'
# @ 50, 1000, this was working best during my testing
#best_trees: 50  best_depth: 1000
#1,424,313.5
#Counter({0.0: 4939, 1.0: 225, 2.0: 82})


my_best_trees = 50
my_best_depth = 1000

my_model = RandomForest(n_estimators=my_best_trees, max_depth=my_best_depth)
my_model.fit(x_train, y_train_p2)

# lets look at cost of that final model
y_pred_my_model = my_model.predict(x_train)
print 'best_trees: {}  best_depth: {}'.format(best_trees, best_depth)
print '{:,}'.format(cost(y_train_p2, y_pred_my_model))

# take a look at the per-flutype counts
print collections.Counter(y_pred_my_model)


until a late breaking change I made somewhere, 50, 1000 were the parameters that were working best
 the cost was notably lower than what I am coming up with now
best_trees: 40  best_depth: 800
1,424,313.5
Counter({0.0: 4939, 1.0: 225, 2.0: 82})


In [41]:
print 'so I am going to use what was working previously for me, that is just the way it is going to be'
print 'since those were the values my following testing were based off of'

# hard code in the numbers to make it clear what I am doing
final_model_p2 = RandomForest(n_estimators=50, max_depth=1000)
final_model_p2.fit(x_train, y_train_p2)

# lets look at cost of that final model
y_pred_model = final_model_p2.predict(x_train)
print 'best_trees: {}  best_depth: {}'.format(50, 1000)
print '{:,}'.format(cost(y_train_p2, y_pred_model))

# take a look at the per-flutype counts
print collections.Counter(y_pred_model)



so I am going to use what was working previously for me, that is just the way it is going to be
since those were the values my following testing were based off of
best_trees: 50  best_depth: 1000
1,428,705.0
Counter({0.0: 4940, 1.0: 223, 2.0: 83})


In [47]:
# didn't have time to formally insert into the tuning run a few cells above but now that we have a depth & n_estimators
#  take moment to see if some manual max_feature setting will help out
feature_base = int(np.sqrt(x_train.shape[1]))
for feature_count in range(feature_base-4, feature_base+5):
    rf_model = RandomForest(n_estimators=best_trees, max_depth=best_depth, max_features=feature_count)
    rf_model.fit(x_train, y_train_p2)
    y_pred = rf_model.predict(x_train)
    print 'feature count: {}  cost: {:,}'.format(feature_count, cost(y_train_p2, y_pred)) 


feature count: 5  cost: 1,424,313.5
feature count: 6  cost: 1,452,955.0
feature count: 7  cost: 1,438,432.0
feature count: 8  cost: 1,438,432.0
feature count: 9  cost: 1,424,313.5
feature count: 10  cost: 1,478,600.0
feature count: 11  cost: 1,443,243.5
feature count: 12  cost: 1,448,608.5
feature count: 13  cost: 1,462,682.0


In [43]:
# above seemed to indicate decreasing max_features a bit from the default might help out, toss that in to final
# (or it clearly did in my earlier tests, of course that shifted a bit)
best_max_features = 8
final_model_p2 = RandomForest(n_estimators=best_trees, max_depth=best_depth, max_features=best_max_features)
final_model_p2.fit(x_train, y_train_p2)
# y_pred = final_model_p2.predict(x_train)
y_pred_model = final_model_p2.predict(x_train)

print 'best_trees: {}  best_depth:{}  best_max_features: {}'.format(best_trees, best_depth, best_max_features)
print 'cost now, observerd data: ${:,}'.format(cost(y_train_p2, y_pred_model))

print 'prediction breakdown: {}'.format(collections.Counter(y_pred_model))


best_trees: 40  best_depth:800  best_max_features: 8
cost now, observerd data: $1,438,432.0
prediction breakdown: Counter({0.0: 4940, 1.0: 224, 2.0: 82})


In [44]:
# early test, replaced with written out version

def flu_predict_p2_first_pass(x_test):
    cols = [col for col in x_test.columns if col in final_columns]
    x_test = x_test[cols]
    x_test = x_test.fillna(test.mean())
    x_test_prepped, y_test = prep_data(x_test, y_col_count=0)
    
    print x_test_prepped.shape
    pred = final_model_p2.predict(x_test_prepped)
    
    return pred

predictions_p2 = flu_predict_p2_first_pass(test)
collections.Counter(predictions_p2)

(1533L, 84L)


Counter({0.0: 1520, 1.0: 10, 2.0: 3})

In [45]:
def flu_predict(x_test):
    
    #Encode categorical variables
    def encode_categorical(array):
        if not array.dtype == np.dtype('float64'):
            return preprocessing.LabelEncoder().fit_transform(array) 
        else:
            return array
    
    cols = [col for col in x_test.columns if col in final_columns]
    x_test = x_test[cols]
    x_test = x_test.fillna(test.mean()) 

    to_float = x_test.dtypes[x_test.dtypes == 'int64'].index
    # Converted columns to floating point
    for feature_name in to_float:
        #if feature_name != 'ID':
        x_test[feature_name] = x_test[feature_name].astype(float)

    # Categorical columns for use in one-hot encoder
    categorical = (x_test.dtypes.values != np.dtype('float64'))    
    x_test = x_test.apply(encode_categorical)
   
    x = x_test.values

    # Apply one hot endcoing
    encoder = preprocessing.OneHotEncoder(categorical_features=categorical, sparse=False)  # Last value in mask is y
    x = encoder.fit_transform(x)

    # final model determined via CV & param tuning
    pred = final_model_p2.predict(x)
    
    return pred

# quick check
print collections.Counter(flu_predict(test))

Counter({0.0: 1520, 1.0: 10, 2.0: 3})


In [46]:
# very low prediction counts for has-flu flu strains, no time to re-investigate
predictions_p2 = flu_predict(test)
print collections.Counter(predictions_p2)

# write to file
out = np.column_stack((test['ID'].values, predictions_p2))
np.savetxt('p2_out.csv', out, delimiter=',', header='ID,flutype', comments='')

back_in = pd.read_csv('p2_out.csv', low_memory=False)  
back_in.head()

Counter({0.0: 1520, 1.0: 10, 2.0: 3})


,ID,flutype
0,51625,0
1,51678,0
2,51694,0
3,51695,0
4,51711,0
